# QOSF Screening Task 3

### 0. Task Description

Please write a simple compiler – program, which translates one quantum circuit into another, using a restricted set of gates.

You need to consider just the basic gates for the input circuit, such as (I, H, X, Y, Z, RX, RY, RZ, CNOT, CZ).

The output circuit should consist only from the following gates: RX, RZ, CZ. In other words, each gate in the original circuit must be replaced by an equivalent combination of gates coming from the restricted set (RX, RZ, CZ) only.

For example, a Hadamard gate after compilation looks like this:

RZ(pi/2)
RX(pi/2)
RZ(pi/2)

Analyze what’s the overhead of the compiled program compared to the original one and propose how to improve it. What we mean by overhead is the following: by replacing all the initial gates with the restricted set of gates given in the problem, you will see that the resulting circuit is much more involved than the original one. This is what we called the overhead, and you may think about how to treat this problem, i.e. you could try to simplify as much as possible the resulting circuit.


### 1. Gate Identities

Rotational operators are defined as:

$$
\begin{align}
R_x(\theta) &= \begin{pmatrix} cos(\frac{\theta}{2}) & -isin(\frac{\theta}{2}) \\ -isin(\frac{\theta}{2}) & cos(\frac{\theta}{2}) \end{pmatrix} \\
R_y(\theta) &= \begin{pmatrix} cos(\frac{\theta}{2}) & -sin(\frac{\theta}{2}) \\ sin(\frac{\theta}{2}) & cos(\frac{\theta}{2}) \end{pmatrix} \\
R_z(\theta) &= \begin{pmatrix} e^{-i\frac{\theta}{2}} & 0 \\ 0 & e^{i\frac{\theta}{2}} \end{pmatrix}
\end{align}
$$

It's easy to observe the following identities:

$$
X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}\propto R_x(\pi) \qquad
Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}\propto R_y(\pi) \qquad
Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}\propto R_z(\pi)
$$

where $\propto$ denotes "proportional to", or "**equivalent up to global phase**".

With the famous commutation relations of $X$, $Y$ and $Z$, we can also observe that:

$$
\sigma_i \sigma_j \propto \sigma_j \sigma_i
$$

where $\sigma_i, \sigma_j \in \{I, X, Y, Z\}$

We can also easily express $S$ in terms of $R_z(\theta)$:

$$
S = \begin{pmatrix} 1 & 0 \\ 0 & i \end{pmatrix}\propto R_z(\frac{\pi}{2})
$$

Therefore, we have a way to express $R_y(\theta)$ in terms of $R_x(\theta)$ and $R_z(\theta)$ by using the Clifford property of $S$ gate:

$$
\begin{align}
R_y(\theta) &= SR_x(\theta)S^{\dagger}\\
         &\propto R_z(\frac{\pi}{2})R_x(\theta)R_z(-\frac{\pi}{2})
\end{align}
$$

and also a way to express $Y$:

$$
\begin{align}
Y &\propto R_y(\pi)\\
  &\propto R_z(\frac{\pi}{2})R_x(\pi)R_z(-\frac{\pi}{2})
\end{align}
$$

We can also express $H$ in terms of $R_x(\theta)$ and $R_z(\theta)$:

$$
\begin{align}
H &= \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix} \\
&\propto R_y(\frac{\pi}{2})R_z(\pi) \\
&\propto R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})
\end{align}
$$

Therefore, transforming $CX$ to $CZ$, $R_x(\theta)$ and $R_z(\theta)$ is also realized using the Clifford property of $H$ gate:

$$
\begin{align}
CX_{0,1} &= H_1CZ_{0,1}H_1\\
  &\propto [R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})]_1 CZ_{0,1} [R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})]_1\\
  &\propto [R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})]_1 CZ_{0, 1} [R_z(\pi)R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})]_1\\
\end{align}
$$

We can further observe X gate is equvalent to a 180° rotation around X-axis (or simple X gate) sandwiched by two 90° around Z-axis:

$$
\begin{align}
X &= HZH \\
  &\propto R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})ZR_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})\\
  &\propto R_z(\frac{\pi}{2})Rx(\pi)R_z(\frac{\pi}{2})\\
  &\propto R_z(\frac{\pi}{2})XR_z(\frac{\pi}{2})\\
\end{align}
$$

By symmetry, we have the other 5 equations:

$$
X \propto R_y(\frac{\pi}{2})XR_y(\frac{\pi}{2})\\
Y \propto R_x(\frac{\pi}{2})YR_x(\frac{\pi}{2})\\
Y \propto R_z(\frac{\pi}{2})YR_z(\frac{\pi}{2})\\
Z \propto R_x(\frac{\pi}{2})ZR_x(\frac{\pi}{2})\\
Z \propto R_y(\frac{\pi}{2})ZR_y(\frac{\pi}{2})
$$

Therefore, a new way with one less gate to express $Y$ in terms of $R_x(\theta)$ and $R_z(\theta)$ is:

$$
\begin{align}
Y &\propto R_z(\frac{\pi}{2})R_x(\pi)R_z(-\frac{\pi}{2}) \\
  &\propto R_z(\frac{\pi}{2})R_z(\frac{\pi}{2})R_x(\pi)R_z(-\frac{\pi}{2})R_z(\frac{\pi}{2})\\
  &\propto R_z(\pi)R_x(\pi) \\
\end{align}
$$

Therefore, we have all the identities used for circuit rewriting. Next we will mention several identities that will be useful for circuit optimization. **In this notebook, our main goal is to reduce both the number of gates and circuit depth, and we will focus on  reducing the number of one-qubit gates**.

The two very obvious ones are gate merging:

$$
R_x(\theta_1)R_x(\theta_2) = R_x(\theta_1+\theta_2) \qquad
R_z(\theta_1)R_z(\theta_2) = R_z(\theta_1+\theta_2) 
$$

Two of the equations mentioned before can also be used during circuit optimization. One is:

$$
R_x(\pi) R_z(\pi) \propto R_z(\pi) R_x(\pi) 
$$

which can be used to change order and seek for gate merging. The other is:

$$
R_z(\frac{\pi}{2})R_x(\pi)R_z(\frac{\pi}{2}) \propto R_x(\pi)\\
R_x(\frac{\pi}{2})R_z(\pi)R_x(\frac{\pi}{2}) \propto R_z(\pi)
$$

which can be used to directly reduce the number of gates.

Another identity can be found by sandwiching $R_z(\theta)$ with $H$ gate and expand it into a series of $R_z(\theta)$ and $R_x(\theta)$:

$$
\begin{align}
R_x(\theta) &= HR_z(\theta)H\\
         &\propto R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})R_z(\theta)R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})\\
         &= R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\theta+\pi)R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})
\end{align}
$$

for wich we can do the same for $R_z(\theta)$ and reverse the order to get two very useful identities for circuit optimization:

$$
R_z(\alpha)R_x(\frac{\pi}{2})R_z(\theta)R_x(\frac{\pi}{2})R_z(\beta)\propto R_z(\alpha-\frac{\pi}{2})R_x(\theta-\pi)R_z(\beta-\frac{\pi}{2})\\
R_x(\alpha)R_z(\frac{\pi}{2})R_x(\theta)R_z(\frac{\pi}{2})R_x(\beta)\propto R_x(\alpha-\frac{\pi}{2})R_z(\theta-\pi)R_x(\beta-\frac{\pi}{2})
$$

It's also worth mentioning that $R_z(\theta)$ can freely move across $CZ$ on both control and target qubit:

$$
CZ_{0, 1}R_z(\theta)_0 \propto R_z(\theta)_0 CZ_{0, 1} \\
CZ_{0, 1}R_z(\theta)_1 \propto R_z(\theta)_1 CZ_{0, 1}
$$

and that concludes all the gate identities we'll be using throught the notebook.

### 2. Initialization

This part of the notebook intializes packages and defines global variables.

In [104]:
#initialization
import qiskit
from qiskit import execute, Aer
from qiskit.circuit.library import IGate, XGate, YGate, ZGate, HGate, SGate, RXGate, RYGate, RZGate, CXGate, CZGate
from qiskit.quantum_info import Statevector
import numpy as np
import random

pi = np.pi

GATE_SET = {'i', 'h', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'cx', 'cz'}
BASIS_GATE_SET = {'rx', 'rz', 'cz'}

I = IGate().to_matrix()
X = XGate().to_matrix()
Y = YGate().to_matrix()
Z = ZGate().to_matrix()
H = ZGate().to_matrix()
S = ZGate().to_matrix()
RX = lambda t:RXGate(t).to_matrix()
RY = lambda t:RYGate(t).to_matrix()
RZ = lambda t:RZGate(t).to_matrix()
CX = CXGate().to_matrix()
CZ = CZGate().to_matrix()

### 3. Helper Functions

This part of the notebook defines several helper functions.

In [1017]:
# Helper functions

def presice_matrix(gates):
    # return the matrix of a list of gates in the precision of machine epsilon
    if len(gates) == 0:
        return 1
    return gates[-1].to_matrix().dot(presice_matrix(gates[:-1]))

def remove_global_phase(mat):
    # return the matrix with its global phase removed
    # global phase = the phase of the first non-zero term in first row
    for i in range(len(mat[0])):
        if np.round(mat[0][i], 3) != 0:
            global_phase_angle = np.angle(mat[0][i])
            break
    global_phase = np.e**(global_phase_angle*1j)
    return mat/global_phase

def matrix(gates, precision=3):
    # return the matrix of a list of gates in certain precision
    if precision == None:
        return precise_matrix(gates)
    else:
        return np.round(presice_matrix(gates), precision)

def matrix_ng(gates, precision=3):
    # return the matrix of a list of gates without global phase removed in certain precision
    if len(gates) == 0:
        return 1
    mat = presice_matrix(gates)
    mat = remove_global_phase(mat)
    if precision == None:
        return mat
    else:
        return np.round(mat, precision)

def equal(gates1, gates2):
    # return if two lists of gates are equal
    return (matrix(gates1) == matrix(gates2)).all()

def equal_ng(gates1, gates2):
    # return if two lists of gates are equal up to global phase
    return (matrix_ng(gates1) == matrix_ng(gates2)).all()

def equal_mat(mat1, mat2):
    # return if two matrices are equal
    return (mat1 == mat2).all()

def equal_mat_ng(mat1, mat2):
    # return if two lists of gates are equal up to global phase
    return (remove_global_phase(mat1) == remove_global_phase(mat2)).all()
    
def commute(gates1, gates2):
    # return if two lists of gates commute
    return equal(gates1+gates2, gates2+gates1)

def commute_ng(gates1, gates2):
    # return if two matrices commute up to global phase, or AB = exp(i*theta) BA
    return equal_ng(gates1+gates2, gates2+gates1)

def commute_mat(mat1, mat2):
    # return if two matrices commute
    return equal_mat(mat1.dot(mat2), mat2.dot(mat1))

def commute_mat_ng(mat1, mat2):
    # return if two matrices commute up to global phase, or AB = exp(i*theta) BA
    return equal_mat_ng(mat1.dot(mat2), mat2.dot(mat1))

def gate_to_str(gate):
    # return info about a gate in str format
    return gate.name if len(gate.params)==0 else f'{gate.name}({np.round(gate.params[0]/pi, 3)}pi)'

def equal_circuit(qc1, qc2):
    # return if two circutis are equal
    backend_sim = Aer.get_backend('unitary_simulator')
    job_sim = execute([qc1, qc2], backend_sim)
    result_sim = job_sim.result()
    unitary1 = result_sim.get_unitary(qc1)
    unitary2 = result_sim.get_unitary(qc2)
    return np.allclose(unitary1, unitary2)

def equal_circuit_ng(qc1, qc2):
    # return if two circuits are equal up to global phase
    s1 = Statevector.from_instruction(qc1)
    s2 = Statevector.from_instruction(qc2)
    return s1.equiv(s2)

def equal_test(gates1, gates2):
    # print if two set of gates are equal or euqal up to global phase
    print(f'{[gate_to_str(g) for g in gates1]} vs {[gate_to_str(g) for g in gates2]}')
    print(f'equal: {equal(gates1, gates2)}')
    print(f'equal up to global phase: {equal_ng(gates1, gates2)}')
    print()

def commute_test(gates1, gates2):
    # print if two set of gates commute or commute up to global phase
    print(f'{[gate_to_str(g) for g in gates1]} vs {[gate_to_str(g) for g in gates2]}')
    print(f'commute: {commute(gates1, gates2)}')
    print(f'commute up to global phase: {commute_ng(gates1, gates2)}')
    print()
    
def equal_circuit_test(qc1, qc2):
    # print if two circuits are equal or equal up to global phase
    print(f'{qc1.name} vs {qc2.name}')
    print(f'equal: {equal_circuit(qc1, qc2)}')
    print(f'equal up to global phase: {equal_circuit_ng(qc1, qc2)}')
    print()
    
def compare_circuits(qcs):
    # print the depth and number of gates of a list of circuits
    print('circuit depth:')
    for qc in qcs:
        print(f'{qc.name}: {qc.depth()}')
    print()
    print('number of gates:')
    for qc in qcs:
        print(f'{qc.name}: {len(qc)}')

### 4. Identity Verification

We then use the helper functions defined above to verify some of the gate identies derived in the first section.

In [1018]:
gates1 = [XGate()]
gates2 = [ZGate()]
commute_test(gates1, gates2)

gates1 = [YGate()]
gates2 = [XGate(), ZGate()]
equal_test(gates1, gates2)

gates1 = [RZGate(-pi/2), RXGate(pi), RZGate(pi/2)]
gates2 = [XGate(), ZGate()]
equal_test(gates1, gates2)

gates1 = [XGate()]
gates2 = [RXGate(pi)]
equal_test(gates1, gates2)

gates1 = [HGate()]
gates2 = [RZGate(pi), RYGate(pi/2)]
equal_test(gates1, gates2)

gates1 = [HGate()]
gates2 = [RZGate(pi/2), RXGate(pi/2), RZGate(pi/2)]
equal_test(gates1, gates2)

gates1 = [RYGate(1.2345)]
gates2 = [RZGate(-pi/2), RXGate(1.2345), RZGate(pi/2)]
equal_test(gates1, gates2)

gates1 = [RXGate(5.4338)]
gates2 = [RXGate(2.2922), RXGate(pi)]
equal_test(gates1, gates2)

gates1 = [RXGate(1.2345)]
gates2 = [RZGate(pi/2), RXGate(pi/2), RZGate(1.2345+pi), RXGate(pi/2), RZGate(pi/2)]
equal_test(gates1, gates2)

gates1 = [RZGate(1.2345)]
gates2 = [RXGate(pi/2), RZGate(pi/2), RXGate(1.2345+pi), RZGate(pi/2), RXGate(pi/2)]
equal_test(gates1, gates2)

gates1 = [RYGate(pi/2), XGate(), RYGate(pi/2)]
gates2 = [XGate()]
equal_test(gates1, gates2)

gates1 = [RZGate(pi/2), XGate(), RZGate(pi/2)]
gates2 = [XGate()]
equal_test(gates1, gates2)

gates1 = [RXGate(pi/2), YGate(), RXGate(pi/2)]
gates2 = [YGate()]
equal_test(gates1, gates2)

gates1 = [RZGate(pi/2), YGate(), RZGate(pi/2)]
gates2 = [YGate()]
equal_test(gates1, gates2)

gates1 = [RXGate(pi/2), ZGate(), RXGate(pi/2)]
gates2 = [ZGate()]
equal_test(gates1, gates2)

gates1 = [RYGate(pi/2), ZGate(), RYGate(pi/2)]
gates2 = [ZGate()]
equal_test(gates1, gates2)

print(commute_mat_ng(np.kron(I, RZ(5.4321)), CZ))
print(commute_mat_ng(np.kron(RZ(5.4321), I), CZ))

['x'] vs ['z']
commute: False
commute up to global phase: True

['y'] vs ['x', 'z']
equal: False
equal up to global phase: True

['rz(-0.5pi)', 'rx(1.0pi)', 'rz(0.5pi)'] vs ['x', 'z']
equal: False
equal up to global phase: True

['x'] vs ['rx(1.0pi)']
equal: False
equal up to global phase: True

['h'] vs ['rz(1.0pi)', 'ry(0.5pi)']
equal: False
equal up to global phase: True

['h'] vs ['rz(0.5pi)', 'rx(0.5pi)', 'rz(0.5pi)']
equal: False
equal up to global phase: True

['ry(0.393pi)'] vs ['rz(-0.5pi)', 'rx(0.393pi)', 'rz(0.5pi)']
equal: True
equal up to global phase: True

['rx(1.73pi)'] vs ['rx(0.73pi)', 'rx(1.0pi)']
equal: True
equal up to global phase: True

['rx(0.393pi)'] vs ['rz(0.5pi)', 'rx(0.5pi)', 'rz(1.393pi)', 'rx(0.5pi)', 'rz(0.5pi)']
equal: False
equal up to global phase: True

['rz(0.393pi)'] vs ['rx(0.5pi)', 'rz(0.5pi)', 'rx(1.393pi)', 'rz(0.5pi)', 'rx(0.5pi)']
equal: False
equal up to global phase: True

['ry(0.5pi)', 'x', 'ry(0.5pi)'] vs ['x']
equal: True
equal up to glo

### 5. Random Circuit Generator

This function can generate a random circuit given number of qubits, number of gates and a restricted set of gates.

In [1019]:
def random_qc(num_q, num_g, gate_set):
    # create a random circuit with num_q qubits, num_g gates, and a restricted set of gates gate_set
    qc = qiskit.QuantumCircuit(num_q)
    for _ in range(num_g):
        g_name = random.choice(list(gate_set))
        if g_name[0] == 'c':
            c_index, t_index = random.sample(range(num_q), 2)
            getattr(qc, g_name)(c_index, t_index)
        elif g_name[0] == 'r':
            index = random.randint(0, num_q-1)
            param = random.uniform(-2*pi, 2*pi)
            getattr(qc, g_name)(param, index)
        else:
            index = random.randint(0, num_q-1)
            getattr(qc, g_name)(index)
    return qc

### 6. Circuit Rewriting

We rewrite the circuit by replacing each of the gate with their expressions in $\{R_x, R_z, CZ\}$. A complete set of rewriting rules:

$$
\begin{align}
I &\rightarrow {Wire}\\
H &\rightarrow R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})\\
X &\rightarrow R_x(\pi)\\
Y &\rightarrow R_y(\pi)\\
Z &\rightarrow R_z(\pi)\\
R_x(\theta) &\rightarrow R_x(\theta)\\
R_y(\theta) &\rightarrow R_z(\frac{\pi}{2})R_x(\theta)R_z(-\frac{\pi}{2})\\
R_z(\theta) &\rightarrow R_z(\theta)\\
CX_{0, 1} &\rightarrow [R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})]_1 CZ_{0, 1} [R_z(\pi)R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})]_1\\
CZ_{0, 1} &\rightarrow CZ_{0, 1}
\end{align}
$$

In [1020]:
def rewrite(qc):
    # rewrite the circuit with {Rx, Rz, CZ} only
    output_qc = qiskit.QuantumCircuit(qc.num_qubits)
    for gate_info in qc.data:
        gate = gate_info[0]
        g_name = gate.name
        index_list = gate_info[1]
        if len(index_list) == 1:
            index = index_list[0]
            if g_name == 'i':
                pass
            elif g_name == 'h':
                output_qc.rz(pi/2, index)
                output_qc.rx(pi/2, index)
                output_qc.rz(pi/2, index)
            elif g_name == 'x':
                output_qc.rx(pi, index)
            elif g_name == 'y':
                output_qc.rx(pi, index)
                output_qc.rz(pi, index)
            elif g_name == 'z':
                output_qc.rz(pi, index)
            elif g_name == 'rx':
                output_qc.rx(gate.params[0], index)
            elif g_name == 'ry':
                output_qc.rz(-pi/2, index)
                output_qc.rx(gate.params[0], index)
                output_qc.rz(pi/2, index)
            elif g_name == 'rz':
                output_qc.rz(gate.params[0], index)
        else:
            c_index, t_index = index_list
            if g_name == 'cx':
                output_qc.rz(pi/2, t_index)
                output_qc.rx(pi/2, t_index)
                output_qc.rz(pi/2, t_index)
                output_qc.rz(pi/2, t_index)
                output_qc.cz(c_index, t_index)
                output_qc.rx(pi/2, t_index)
                output_qc.rz(pi/2, t_index)
            elif g_name == 'cz':
                output_qc.cz(c_index, t_index)
    return output_qc

### 7. Circuit Optimization

In [1021]:
def clean_gates(qc):
    # remove all Rx and Rz gates with parameter as mutiples of 2pi
    # ensure parameters in the range (-pi, pi)
    output_qc = qiskit.QuantumCircuit(qc.num_qubits)
    num_gates = len(qc.data)
    for i in range(num_gates):
        gate_info_i = qc.data[i]
        index_list_i = gate_info_i[1]
        gate_i = gate_info_i[0]
        g_name_i = gate_i.name
        
        if g_name_i in {'rz', 'rx'}:
            param = gate_i.params[0]
            if np.round(param/(2*pi), 3).is_integer():
                pass
            elif param > pi:
                r = param%(2*pi)
                getattr(output_qc, g_name_i)(r if 0<=r<=pi else r-2*pi, index_list_i[0])
            elif param < -pi:
                r = param%(-2*pi)
                getattr(output_qc, g_name_i)(r if -pi<r<=0 else r+2*pi, index_list_i[0])
            else:
                getattr(output_qc, g_name_i)(param, index_list_i[0])
        elif g_name_i == 'cz':
            getattr(output_qc, g_name_i)(*index_list_i)
    
    return output_qc

def merge_once(qc):
    # Merge adjacent gates with same type once.
    output_qc = qiskit.QuantumCircuit(qc.num_qubits)
    num_gates = len(qc.data)
    check_list = []
    for i in range(num_gates):
        if i in check_list:
            continue
            
        gate_info_i = qc.data[i]
        index_list_i = gate_info_i[1]
        gate_i = gate_info_i[0]
        g_name_i = gate_i.name

        if g_name_i in {'rz', 'rx'}:
            index_i = index_list_i[0]

            for j in range(i+1, num_gates):

                gate_info_j = qc.data[j]
                index_list_j = gate_info_j[1]
                gate_j = gate_info_j[0]
                g_name_j = gate_j.name

                if index_i not in index_list_j:
                    continue
                
                if g_name_j == 'cz':
                    if g_name_i == 'rz':
                        pass
                    elif g_name_i == 'rx':
                        getattr(output_qc, g_name_i)(gate_i.params[0], index_i)
                        break
                elif g_name_j == g_name_i:
                    param_sum = gate_i.params[0]+gate_j.params[0]
                    getattr(output_qc, g_name_i)(param_sum, index_i)
                    check_list.append(j)
                    break
                else:
                    getattr(output_qc, g_name_i)(gate_i.params[0], index_i)
                    break
            else:
                getattr(output_qc, g_name_i)(gate_i.params[0], index_i)

        elif g_name_i == 'cz':
            for j in range(i+1, num_gates):

                gate_info_j = qc.data[j]
                index_list_j = gate_info_j[1]
                gate_j = gate_info_j[0]
                g_name_j = gate_j.name

                set_i = set(index_list_i)
                set_j = set(index_list_j)

                if set_j.intersection(set_i) == set():
                    continue
                if set_i == set_j:
                    check_list.append(j)
                    break
                else:
                    getattr(output_qc, g_name_i)(*index_list_i)
                    break
            else:
                getattr(output_qc, g_name_i)(*index_list_i)
    return output_qc


def merge_all(qc):
    # repeart the merging process untill the circuit is optimal
    old_qc = qc
    new_qc = clean_gates(merge_once(old_qc))
    while len(new_qc.data) < len(old_qc.data):
        old_qc = new_qc
        new_qc = clean_gates(merge_once(old_qc))
    return new_qc

def reorder_once(qc, check_list=[]):
    # swap the position of adjacent Rx(pi) and Rz(pi) once
    output_qc = qc.copy()
    num_gates = len(qc.data)
    checked = None
    for i in range(num_gates):
        if i in check_list:
            continue
        gate_info_i = output_qc.data[i]
        index_list_i = gate_info_i[1]
        gate_i = gate_info_i[0]
        g_name_i = gate_i.name

        if g_name_i not in {'rz', 'rx'}:
            continue
        
        temp = np.round(gate_i.params[0]/pi, 3)
        if temp.is_integer() and int(temp)%2 == 1:
            index_i = index_list_i[0]
            for j in range(i+1, num_gates):

                gate_info_j = output_qc.data[j]
                index_list_j = gate_info_j[1]
                gate_j = gate_info_j[0]
                g_name_j = gate_j.name

                if index_i in index_list_j:
                    if len(index_list_j)==1 and g_name_i != g_name_j:
                        temp = np.round(gate_j.params[0]/pi, 3) 
                        if temp.is_integer() and int(temp)%2 == 1:
                            output_qc.data[i], output_qc.data[j] = output_qc.data[j], output_qc.data[i]
                            checked = i
                    break
            if checked != None:
                break
                    
    return output_qc, checked

def reorder_and_merge_all(qc):
    # Try if the swap makes further optimization possible. Merge all if possible.
    current_qc = merge_all(qc)
    re_qc, checked = reorder_once(current_qc)
    check_list = []
    while checked != None:
        mer_re_qc = merge_all(re_qc)
        if len(mer_re_qc.data) < len(current_qc.data):
            current_qc = mer_re_qc
            re_qc, checked = reorder_once(current_qc)
        else:
            check_list.append(checked)
            re_qc, checked = reorder_once(current_qc, check_list)
            
    return current_qc

        
def find_next_n_gates(qc, i, n):
    # find the next n gates after a given gate in a circuit
    gate_infos = qc.data
    gate_list = []
    for j in range(i+1, len(gate_infos)):
        set_i = set(gate_infos[i][1])
        set_j = set(gate_infos[j][1])
        if set_j.intersection(set_i) != set():
            gate_list.append(j)
        if len(gate_list) == n:
            return gate_list
    
    while len(gate_list) != n:
        gate_list.append(None)
    return gate_list


def group_and_merge_all(qc):
    
    # number: inherent index_list: index of param_tuple
    # tuple: inherent parameter: (index of param_tuple, shift)
    
    name_tuple_list = [('rx', 'rz', 'rx', 'rz', 'rx'),
                       ('rz', 'rx', 'rz', 'rx', 'rz'),
                       ('rz', 'cz', 'rx', 'rz', 'rx', 'rz'),
                       ('rz', 'rx', 'rz', 'rx', 'cz', 'rz'),
                       ('rz', 'cz', 'rx', 'rz', 'rx', 'cz', 'rz')]
    
    param_tuple_list = [(None, pi/2, None, pi/2, None),
                       (None, pi/2, None, pi/2, None),
                       (None, None, pi/2, None, pi/2, None),
                       (None, pi/2, None, pi/2, None, None),
                       (None, None, pi/2, None, pi/2, None, None)]
    
    r_name_tuple_list = [('rx', 'rz', 'rx'),
                          ('rz', 'rx', 'rz'),
                          ('cz', 'rz', 'rx', 'rz'),
                          ('rz', 'rx', 'rz', 'cz'),
                          ('cz', 'rz', 'rx', 'rz', 'cz')]
    
    r_param_tuple_list = [((0, -pi/2), (2, -pi), (4, -pi/2)),
                          ((0, -pi/2), (2, -pi), (4, -pi/2)),
                          (1, (0, -pi/2), (3, -pi), (5, -pi/2)),
                          ((0, -pi/2), (2, -pi), (5, -pi/2), 4),
                          (1, (0, -pi/2), (3, -pi), (6, -pi/2), 5)]
    
    for r_index in range(len(name_tuple_list)):
        
        check_set = set()
        output_qc = qiskit.QuantumCircuit(qc.num_qubits)
        num_gates = len(qc.data)
    
        name_tuple = name_tuple_list[r_index]
        param_tuple = param_tuple_list[r_index]

        r_name_tuple = r_name_tuple_list[r_index]
        r_param_tuple = r_param_tuple_list[r_index]
        
        for i in range(num_gates):
            
            if i in check_set:
                continue
            
            pos_list = [i]+find_next_n_gates(qc, i, len(name_tuple)-1)
            if None in pos_list:
                pass
            else:
                
                i_name_tuple = tuple(qc.data[p][0].name for p in pos_list)
                i_param_tuple = tuple(None if (param_tuple[j] == None or len(qc.data[pos_list[j]][0].params) == 0) \
                                      else qc.data[pos_list[j]][0].params[0] for j in range(len(param_tuple)))

                if i_name_tuple == name_tuple and i_param_tuple == param_tuple:
                    for j in range(len(r_name_tuple)):
                        n = r_name_tuple[j]
                        p = r_param_tuple[j]
                        if type(p) == int and n == 'cz':
                            pos = pos_list[p]
                            for k in range(pos_list[p-1]+1, pos):
                                getattr(output_qc, qc.data[k][0].name)(*qc.data[k][0].params, *qc.data[k][1])
                                check_set.add(k)
                            getattr(output_qc, 'cz')(*qc.data[pos][1])
                        elif type(p) == tuple:
                            getattr(output_qc, n)(qc.data[pos_list[p[0]]][0].params[0]+p[1], qc.data[i][1][0])
                    check_set = check_set.union(set(pos_list))
                    continue
            
            gate_info = qc.data[i]
            gate = gate_info[0]
            gate_name = gate.name
            index_list = gate_info[1]
            
            if gate_name in {'rx', 'rz'}:
                getattr(output_qc, gate_name)(gate.params[0], *index_list)
            elif gate_name == 'cz':
                getattr(output_qc, gate_name)(*index_list)
                
        qc = merge_all(output_qc)

    return output_qc

In [1010]:
qc0 = random_qc(3, 200, GATE_SET)
qc0.rz(10, 0)
qc0.rz(-10, 1)
qc0.draw()

┌───┐     ┌────────────┐┌───┐             ┌───┐     ┌───┐»
q_0: ────┤ X ├─────┤ RZ(4.2474) ├┤ H ├─────■───────┤ X ├─────┤ I ├»
         └─┬─┘     ├────────────┤└───┘   ┌─┴─┐     └─┬─┘     └───┘»
q_1: ──────■───────┤ RX(1.8623) ├──────■─┤ X ├───────■────────────»
     ┌────────────┐└───┬───┬────┘┌───┐ │ ├───┤┌─────────────┐┌───┐»
q_2: ┤ RX(2.3898) ├────┤ H ├─────┤ Z ├─■─┤ X ├┤ RX(-2.8839) ├┤ Y ├»
     └────────────┘    └───┘     └───┘   └───┘└─────────────┘└───┘»
«                                             ┌───┐      ┌─────────────┐     »
«q_0: ─────────────────────────────────────■──┤ Y ├─■──■─┤ RZ(-4.7893) ├─────»
«                                          │  └───┘ │  │ └─────────────┘┌───┐»
«q_1: ─────────────────────────────────────┼────────■──┼────────■───────┤ X ├»
«     ┌─────────────┐┌─────────────┐┌───┐┌─┴─┐         │      ┌─┴─┐     └───┘»
«q_2: ┤ RX(-1.6788) ├┤ RX(-5.7956) ├┤ Z ├┤ X ├─────────■──────┤ X ├──────────»
«     └─────────────┘└─────────────┘└───┘└───┘                └───┘          »
«                                 ┌─────────────┐                     ┌───┐»
«q_0: ─────────────────────────■──┤ RX(-3.4741) ├───────────────────■─┤ I ├»
«        ┌─────────────┐┌───┐  │  └┬────────────┤     ┌───┐┌───┐    │ └───┘»
«q_1: ─■─┤ RY(-3.4631) ├┤ H ├──┼───┤ RZ(-1.618) ├─────┤ X ├┤ Y ├─■──■───■──»
«      │ └────┬───┬────┘├───┤┌─┴─┐ └───┬───┬────┘┌───┐└─┬─┘└───┘ │      │  »
«q_2: ─■──────┤ Y ├─────┤ H ├┤ X ├─────┤ X ├─────┤ H ├──■────────■──────■──»
«             └───┘     └───┘└───┘     └───┘     └───┘                     »
«      ┌────────────┐          ┌────────────┐    ┌───┐     ┌─────────────┐   »
«q_0: ─┤ RY(3.0492) ├───────■──┤ RX(3.8707) ├────┤ H ├─────┤ RY(-2.0964) ├───»
«      ├────────────┤┌───┐  │  └───┬───┬────┘    └───┘     └─────────────┘   »
«q_1: ─┤ RY(2.3376) ├┤ Y ├──┼──────┤ X ├───────────────────────────────────■─»
«     ┌┴────────────┤└───┘┌─┴─┐    ├───┤     ┌────────────┐     ┌───┐      │ »
«q_2: ┤ RZ(0.64374) ├─────┤ X ├────┤ I ├─────┤ RY(2.2368) ├─────┤ I ├──────■─»
«     └─────────────┘     └───┘    └───┘     └────────────┘     └───┘        »
«     ┌───┐┌───┐┌─────────────┐┌─────────────┐                                »
«q_0: ┤ X ├┤ X ├┤ RY(-4.2096) ├┤ RZ(-3.0404) ├──────────────────────────────■─»
«     └─┬─┘├───┤└────┬───┬────┘└────┬───┬────┘┌───┐    ┌───┐     ┌───┐┌───┐ │ »
«q_1: ──┼──┤ X ├─────┤ Z ├──────────┤ X ├─────┤ H ├────┤ I ├─────┤ Z ├┤ I ├─┼─»
«       │  ├───┤     ├───┤          ├───┤     ├───┤┌───┴───┴────┐└───┘└───┘ │ »
«q_2: ──■──┤ Z ├─────┤ X ├──────────┤ I ├─────┤ X ├┤ RZ(4.9187) ├───────────■─»
«          └───┘     └───┘          └───┘     └───┘└────────────┘             »
«     ┌───┐     ┌───┐                    ┌───┐                              »
«q_0: ┤ Y ├─────┤ Y ├────────────────────┤ X ├──────────────────────────────»
«     ├───┤     ├───┤     ┌─────────────┐└─┬─┘┌─────────────┐┌───┐┌───┐┌───┐»
«q_1: ┤ Y ├─────┤ H ├─────┤ RX(-5.6722) ├──┼──┤ RZ(-4.8816) ├┤ Z ├┤ I ├┤ I ├»
«     ├───┤┌────┴───┴────┐└─────────────┘  │  └─────────────┘└───┘└───┘└───┘»
«q_2: ┤ Z ├┤ RZ(-4.7143) ├─────────────────■────────────────────────────────»
«     └───┘└─────────────┘                                                  »
«                                    ┌───┐     ┌────────────┐┌───┐»
«q_0: ───────────────────■───────────┤ X ├─────┤ RX(2.9446) ├┤ Z ├»
«     ┌────────────┐     │  ┌───┐    └─┬─┘     └───┬───┬────┘├───┤»
«q_1: ┤ RX(3.8192) ├─■───■──┤ Z ├──────■───────────┤ I ├─────┤ H ├»
«     └────────────┘ │ ┌───┐├───┤┌────────────┐    └───┘     └───┘»
«q_2: ───────────────■─┤ Z ├┤ I ├┤ RX(0.5213) ├───────────────────»
«                      └───┘└───┘└────────────┘                   »
«     ┌────────────┐┌────────────┐┌───┐                                    »
«q_0: ┤ RZ(3.1804) ├┤ RY(4.7551) ├┤ H ├─────────────────────────────────■──»
«     ├────────────┤└───┬───┬────┘├───┤ ┌────────────┐ ┌─────────────┐  │  »
«q_1: ┤ RY(2.4316) ├────┤ Z ├─────┤ X ├─┤ RY(1.3108) ├─┤ RZ(0.19045) ├──┼──

In [1011]:
qc1 = rewrite(qc0)
qc1.draw()

┌──────────┐ ┌──────────┐┌──────────┐┌──────────┐           ┌──────────┐ »
q_0: ─┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RZ(pi/2) ├────■──────┤ RX(pi/2) ├─»
      └──────────┘ └──────────┘└──────────┘└──────────┘    │     ┌┴──────────┴┐»
q_1: ──────────────────────────────────────────────────────■─────┤ RX(1.8623) ├»
     ┌────────────┐┌──────────┐┌──────────┐┌──────────┐┌────────┐└────────────┘»
q_2: ┤ RX(2.3898) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RZ(pi) ├──────────────»
     └────────────┘└──────────┘└──────────┘└──────────┘└────────┘              »
«     ┌──────────┐┌────────────┐  ┌──────────┐ ┌──────────┐┌──────────┐»
«q_0: ┤ RZ(pi/2) ├┤ RZ(4.2474) ├──┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├»
«     └──────────┘└┬──────────┬┘  ├──────────┤ ├──────────┤├──────────┤»
«q_1: ─────■───────┤ RZ(pi/2) ├───┤ RX(pi/2) ├─┤ RZ(pi/2) ├┤ RZ(pi/2) ├»
«          │       └┬────────┬┘ ┌─┴──────────┴┐└┬────────┬┘└┬────────┬┘»
«q_2: ─────■────────┤ RX(pi) ├──┤ RX(-2.8839) ├─┤ RX(pi) ├──┤ RZ(pi) ├─»
«                   └────────┘  └─────────────┘ └────────┘  └────────┘ »
«                      ┌──────────┐ ┌──────────┐┌──────────┐┌──────────┐»
«q_0: ───────■─────────┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RZ(pi/2) ├»
«            │         ├──────────┤ ├──────────┤└──────────┘└──────────┘»
«q_1: ───────■─────────┤ RX(pi/2) ├─┤ RZ(pi/2) ├────────────────────────»
«     ┌─────────────┐┌─┴──────────┴┐└┬────────┬┘┌──────────┐┌──────────┐»
«q_2: ┤ RX(-1.6788) ├┤ RX(-5.7956) ├─┤ RZ(pi) ├─┤ RZ(pi/2) ├┤ RX(pi/2) ├»
«     └─────────────┘└─────────────┘ └────────┘ └──────────┘└──────────┘»
«                 ┌──────────┐┌──────────┐    ┌────────┐  ┌────────┐       »
«q_0: ─────■──────┤ RX(pi/2) ├┤ RZ(pi/2) ├─■──┤ RX(pi) ├──┤ RZ(pi) ├──■──■─»
«          │      └──────────┘└──────────┘ │  └────────┘  └────────┘  │  │ »
«q_1: ─────■───────────────────────────────┼──────────────────────────■──┼─»
«     ┌──────────┐┌──────────┐             │ ┌──────────┐┌──────────┐    │ »
«q_2: ┤ RZ(pi/2) ├┤ RZ(pi/2) ├─────────────■─┤ RX(pi/2) ├┤ RZ(pi/2) ├────■─»
«     └──────────┘└──────────┘               └──────────┘└──────────┘      »
«     ┌─────────────┐                                                   »
«q_0: ┤ RZ(-4.7893) ├───────────────────────────────────────────────────»
«     └─────────────┘                                        ┌────────┐ »
«q_1: ────────────────────────────────────────────────────■──┤ RX(pi) ├─»
«       ┌──────────┐ ┌──────────┐┌──────────┐┌──────────┐ │ ┌┴────────┴┐»
«q_2: ──┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RZ(pi/2) ├─■─┤ RX(pi/2) ├»
«       └──────────┘ └──────────┘└──────────┘└──────────┘   └──────────┘»
«                                                                        »
«q_0: ───────────────────────────────────────────────────────────────────»
«                    ┌───────────┐┌─────────────┐┌──────────┐┌──────────┐»
«q_1: ─────────────■─┤ RZ(-pi/2) ├┤ RX(-3.4631) ├┤ RZ(pi/2) ├┤ RZ(pi/2) ├»
«     ┌──────────┐ │ └─┬────────┬┘└──┬────────┬─┘├──────────┤├──────────┤»
«q_2: ┤ RZ(pi/2) ├─■───┤ RX(pi) ├────┤ RZ(pi) ├──┤ RZ(pi/2) ├┤ RX(pi/2) ├»
«     └──────────┘     └────────┘    └────────┘  └──────────┘└──────────┘»
«                                                                      »
«q_0: ───────────────────────────────────────────────────────────────■─»
«     ┌──────────┐┌──────────┐┌────────────┐┌──────────┐┌──────────┐ │ »
«q_1: ┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RZ(-1.618) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├─┼─»
«     ├──────────┤├──────────┤└┬──────────┬┘├──────────┤├──────────┤ │ »
«q_2: ┤ RZ(pi/2) ├┤ RZ(pi/2) ├─┤ RX(pi/2) ├─┤ RZ(pi/2) ├┤ RZ(pi/2) ├─■─»
«     └──────────┘└──────────┘ └──────────┘ └──────────┘└──────────┘   »
«     ┌─────────────┐                                                          »
«q_0: ┤ RX(-3.4741) ├──────────────────────────────────────────────────────────»
«     └─┬──────────┬┘┌──────────┐                                              »
«q_1: ──┤ RZ(pi/2) ├─┤ RZ(pi/2) ├─────────────────────────────────────────

In [1012]:
qc2 = merge_all(qc1)
qc2.draw()

┌──────────┐ ┌──────────┐ ┌────────┐               ┌──────────┐ »
q_0: ─┤ RZ(pi/2) ├─┤ RX(pi/2) ├─┤ RZ(pi) ├───────■───────┤ RX(pi/2) ├─»
      └──────────┘ └──────────┘ └────────┘       │      ┌┴──────────┴┐»
q_1: ────────────────────────────────────────────■──────┤ RX(1.8623) ├»
     ┌────────────┐┌──────────┐┌──────────┐┌───────────┐└────────────┘»
q_2: ┤ RX(2.3898) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(-pi/2) ├──────────────»
     └────────────┘└──────────┘└──────────┘└───────────┘              »
«     ┌────────────┐  ┌──────────┐  ┌────────┐                             »
«q_0: ┤ RZ(1.1058) ├──┤ RX(pi/2) ├──┤ RZ(pi) ├──────────────────────■──────»
«     └────────────┘  ├──────────┤ ┌┴────────┴┐   ┌────────┐        │      »
«q_1: ──────■─────────┤ RZ(pi/2) ├─┤ RX(pi/2) ├───┤ RZ(pi) ├────────■──────»
«           │       ┌─┴──────────┴┐└┬────────┬┘┌──┴────────┴─┐┌───────────┐»
«q_2: ──────■───────┤ RX(-2.8839) ├─┤ RZ(pi) ├─┤ RX(-1.1913) ├┤ RZ(-pi/2) ├»
«                   └─────────────┘ └────────┘ └─────────────┘└───────────┘»
«     ┌──────────┐ ┌────────┐    ┌──────────┐┌──────────┐    ┌────────┐ »
«q_0: ┤ RX(pi/2) ├─┤ RZ(pi) ├──■─┤ RX(pi/2) ├┤ RZ(pi/2) ├─■──┤ RX(pi) ├─»
«     ├──────────┤┌┴────────┴┐ │ └──────────┘└──────────┘ │  └────────┘ »
«q_1: ┤ RX(pi/2) ├┤ RZ(pi/2) ├─■──────────────────────────┼─────────────»
«     ├──────────┤└┬────────┬┘                            │ ┌──────────┐»
«q_2: ┤ RX(pi/2) ├─┤ RZ(pi) ├─────────────────────────────■─┤ RX(pi/2) ├»
«     └──────────┘ └────────┘                               └──────────┘»
«     ┌─────────────┐                                                          »
«q_0: ┤ RZ(-1.6477) ├─■──■─────────────────────────────────────────────────────»
«     └─────────────┘ │  │                           ┌────────┐                »
«q_1: ────────────────■──┼────────────────────────■──┤ RX(pi) ├──────────────■─»
«        ┌────────┐      │ ┌──────────┐┌────────┐ │ ┌┴────────┴┐┌──────────┐ │ »
«q_2: ───┤ RZ(pi) ├──────■─┤ RX(pi/2) ├┤ RZ(pi) ├─■─┤ RX(pi/2) ├┤ RZ(pi/2) ├─■─»
«        └────────┘        └──────────┘└────────┘   └──────────┘└──────────┘   »
«                                                                      »
«q_0: ─────────────────────────────────────────────────────────────────»
«     ┌───────────┐┌────────────┐ ┌────────┐ ┌──────────┐┌────────────┐»
«q_1: ┤ RZ(-pi/2) ├┤ RX(2.8201) ├─┤ RZ(pi) ├─┤ RX(pi/2) ├┤ RZ(1.5236) ├»
«     └─┬────────┬┘├───────────┬┘┌┴────────┴┐└┬────────┬┘└┬──────────┬┘»
«q_2: ──┤ RX(pi) ├─┤ RZ(-pi/2) ├─┤ RX(pi/2) ├─┤ RZ(pi) ├──┤ RX(pi/2) ├─»
«       └────────┘ └───────────┘ └──────────┘ └────────┘  └──────────┘ »
«                    ┌────────────┐                                  »
«q_0: ─────────────■─┤ RX(2.8091) ├──────────────────────────────────»
«     ┌──────────┐ │ └─┬────────┬─┘                                  »
«q_1: ┤ RX(pi/2) ├─┼───┤ RZ(pi) ├────────────────────────────────────»
«     └┬────────┬┘ │  ┌┴────────┴┐ ┌──────────┐┌────────┐┌──────────┐»
«q_2: ─┤ RZ(pi) ├──■──┤ RX(pi/2) ├─┤ RZ(pi/2) ├┤ RX(pi) ├┤ RZ(pi/2) ├»
«      └────────┘     └──────────┘ └──────────┘└────────┘└──────────┘»
«                                                                     »
«q_0: ────────────────────────────────────────────────────────────────»
«                                   ┌──────────┐┌──────────┐┌────────┐»
«q_1: ────────────────────────────■─┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RX(pi) ├»
«     ┌──────────┐┌─────────────┐ │ └──────────┘└──────────┘└────────┘»
«q_2: ┤ RX(pi/2) ├┤ RZ(-2.4979) ├─■───────────────────────────────────»
«     └──────────┘└─────────────┘                                     »
«                       ┌───────────┐┌────────────┐┌──────────┐             »
«q_0: ────────────────■─┤ RZ(-pi/2) ├┤ RX(3.0492) ├┤ RZ(pi/2) ├───────────■─»
«     ┌──────────┐    │ └───────────┘├────────────┤├──────────┤┌────────┐ │ »
«q_1: ┤ RZ(pi/2) ├─■──■───────■──────┤ RX(2.3376) ├┤ RZ(pi/2) ├┤ RX(pi) ├─┼─»
«     └──────────┘ │          │      └┬──────────┬┘└┬─

In [1013]:
qc3 = reorder_and_merge_all(qc2)
#qc3.draw()

In [1014]:
qc4 = group_and_merge_all(qc3)
#qc4.draw()

In [1015]:
equal_circuit_test(qc0, qc1)
equal_circuit_test(qc1, qc2)
equal_circuit_test(qc2, qc3)
equal_circuit_test(qc3, qc4)

circuit39939 vs circuit39940
equal: False
equal up to global phase: True

circuit39940 vs circuit39948
equal: True
equal up to global phase: True

circuit39948 vs circuit39974
equal: True
equal up to global phase: True

circuit39974 vs circuit39999
equal: False
equal up to global phase: True



In [1016]:
compare_circuits([qc0, qc1, qc2, qc3, qc4])

circuit depth:
circuit39939: 116
circuit39940: 210
circuit39948: 152
circuit39974: 148
circuit39999: 131

number of gates:
circuit39939: 202
circuit39940: 398
circuit39948: 272
circuit39974: 255
circuit39999: 225
